In [2]:
import openpyxl as px
import numpy as np
import pandas as pd
import geopy.distance

In [3]:
# parsing the data from the excel file

file = '../data/SoilCdataALcottonfield.xlsx'
wb_obj = px.load_workbook(file, read_only=True, data_only=True)
sheet_obj = wb_obj.active
row = sheet_obj.max_row
column = sheet_obj.max_column

# column 1, sample ids
# indexs start at row 14, and are formatted with numbers (location_id) - (sample_id) - (portion_id)
# position ids are A-F
# the row after position F is the entire sample
# the row after the entire sample is the next location

data = np.zeros((25, 5, 6, 6), dtype=object)
sample_ids = []
c_percents = []
n_percents = []
lats = []
lons = []

for location_id_row in range(25):
    lat = sheet_obj.cell(row=location_id_row*35+14, column=2).value
    lon = sheet_obj.cell(row=location_id_row*35+14, column=3).value
    for sample_id_row in range(5):
        for position_id_row in range(6):
            row = location_id_row*35+sample_id_row*7+14+position_id_row
            id_val = sheet_obj.cell(row=row, column=1).value
            c_percent = sheet_obj.cell(row=row, column=8).value
            n_percent = sheet_obj.cell(row=row, column=7).value
            sample_ids.append(id_val)
            c_percents.append(c_percent)
            n_percents.append(n_percent)
            data[location_id_row, sample_id_row, position_id_row, 0] = id_val

            data[location_id_row, sample_id_row, position_id_row, 1] = float(c_percent)
            data[location_id_row, sample_id_row, position_id_row, 2] = float(n_percent)
            data[location_id_row, sample_id_row, position_id_row, 3] = lat
            data[location_id_row, sample_id_row, position_id_row, 4] = lon

# this is an index of the depths of the samples
# A= 0-5; B=5-10; C=10-15; D=15-20; E=20-25; F=25-30
lat_index = 5
data[:, :, 0, lat_index] = -2.5
data[:, :, 1, lat_index] = -7.5
data[:, :, 2, lat_index] = -12.5
data[:, :, 3, lat_index] = -17.5
data[:, :, 4, lat_index] = -22.5
data[:, :, 5, lat_index] = -27.5



np.save('../data/soil_data.npy', data)

In [4]:
df = pd.DataFrame(data.reshape(-1, 6), columns=['id', 'c_percent', 'n_percent', 'lat', 'lon', 'z'])

df['c_percent'] = df['c_percent'].astype(float)
df['n_percent'] = df['n_percent'].astype(float)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)
df['z'] = df['z'].astype(float)
df['site'] = np.repeat(np.arange(25), 5*6)
def sitt(string):
    return string.split('-')[-1]
df['site_alt'] = df['site'].astype(str) + '_' + df['id'].apply(sitt).astype(str)
df['site_alt_id'] = df['id'].apply(sitt).astype(str)

df['lower_bin'] = df['z']-2.5
df['upper_bin'] = df['z']+2.5

average_lat = np.mean(df['lat'])
average_lon = np.mean(df['lon'])

_xs = []
_ys = []
for i in range(len(df)):
    _xs.append(geopy.distance.distance((average_lat, df['lon'][i]), (df['lat'][i], df['lon'][i])).m*100)
    _ys.append(geopy.distance.distance((df['lat'][i], average_lon), (df['lat'][i], df['lon'][i])).m*100)
df['_x'] = _xs
df['_y'] = _ys

# df['x'] = df['_x']
# df['y'] = df['_y']

def get_core(_str):
    return int(_str.split('-')[1])
df['core'] = df['id'].apply(get_core)

def push_core(id, x, y):
    _id = int(id)
    if _id == 1:
        return x, y
    elif _id == 2:
        return x+500, y+500
    elif _id == 3:
        return x+500, y-500
    elif _id == 4:
        return x-500, y+500
    elif _id == 5:
        return x-500, y-500
    
df['x'], df['y'] = zip(*df.apply(lambda x: push_core(x['core'], x['_x'], x['_y']), axis=1))

# center x and y values around the average
df['x'] = df['x'] - np.mean(df['x'])
df['y'] = df['y'] - np.mean(df['y'])

df['_x'] = df['_x'] - np.mean(df['_x'])
df['_y'] = df['_y'] - np.mean(df['_y'])


df.to_csv('../data/soil_data.csv')